<a href="https://colab.research.google.com/github/sidsharma3/TextGenerationDeepLearningModel/blob/main/TextGenerationRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function, division
from builtins import range, input
import numpy as np
import pandas as pd
import os
import sys
import string
from keras.models import Model
from keras.layers import Dense, Embedding, Input, LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import Adam, SGD

In [ ]:
MAX_SEQUENCE_LENGTH = 100
MAX_VOCAB_SIZE = 3000
EMBEDDING_DIM = 100
input_texts = []
target_texts = []
for line in open('./textBase.txt'):
  line = line.rstrip()
  if not line:
    continue
  input_line = '<sos> ' + line
  target_line = line + ' <eos>'
  input_texts.append(input_line)
  target_texts.append(target_line)
all_lines = input_texts + target_texts

In [ ]:
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, filters='')
tokenizer.fit_on_texts(all_lines)
target_sequences = tokenizer.texts_to_sequences(target_texts)
input_sequences = tokenizer.texts_to_sequences(input_texts)
max_sequence_length_from_data = max(len(s) for s in input_sequences)
print('Max sequence length:', max_sequence_length_from_data)

Max sequence length: 12


In [ ]:
word2idx = tokenizer.word_index
# Need to pad sequences to make then equal in shape
max_sequence_length = min(max_sequence_length_from_data, MAX_SEQUENCE_LENGTH)
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='post')
target_sequences = pad_sequences(target_sequences, maxlen=max_sequence_length, padding='post')

In [ ]:
word2vec = {}
with open(os.path.join('/content/drive/MyDrive/machineLearning/glove.6B.100d.txt')) as f:
  # is just a space-separated text file in the format:
  # word vec[0] vec[1] vec[2] ...
  for line in f:
    values = line.split()
    word = values[0]
    vec = np.asarray(values[1:], dtype='float32')
    word2vec[word] = vec
num_words = min(MAX_VOCAB_SIZE, len(word2idx) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word2idx.items():
  if i < MAX_VOCAB_SIZE:
    embedding_vector = word2vec.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector

In [ ]:
one_hot_targets = np.zeros((len(input_sequences), max_sequence_length, num_words))
for i, target_sequence in enumerate(target_sequences):
  for t, word in enumerate(target_sequence):
    if word > 0:
      one_hot_targets[i, t, word] = 1
embedding_layer = Embedding(
  num_words,
  EMBEDDING_DIM,
  weights=[embedding_matrix],
)

In [ ]:
input_ = Input(shape=(max_sequence_length,))
initial_h = Input(shape=(25,))
initial_c = Input(shape=(25,))
x = embedding_layer(input_)
lstm = LSTM(25, return_sequences=True, return_state=True)
x, _, _ = lstm(x, initial_state=[initial_h, initial_c]) # don't need the states here
dense = Dense(num_words, activation='softmax')
output = dense(x)
model = Model([input_, initial_h, initial_c], output)
model.compile(
  loss='categorical_crossentropy',
  optimizer='rmsprop',
  metrics=['accuracy']
)
z = np.zeros((len(input_sequences), 25))
r = model.fit(
  [input_sequences, z, z],
  one_hot_targets,
  batch_size=128,
  epochs=500,
  validation_split=0.2
)

Epoch 1/500
9/9 [==============================] - 4s 190ms/step - loss: 5.5557 - accuracy: 0.0078 - val_loss: 5.5587 - val_accuracy: 0.0081
Epoch 2/500
9/9 [==============================] - 1s 112ms/step - loss: 5.3708 - accuracy: 0.0142 - val_loss: 5.4048 - val_accuracy: 0.0093
Epoch 3/500
9/9 [==============================] - 1s 115ms/step - loss: 5.2174 - accuracy: 0.0151 - val_loss: 5.2789 - val_accuracy: 0.0093
Epoch 4/500
9/9 [==============================] - 1s 115ms/step - loss: 5.1473 - accuracy: 0.0158 - val_loss: 5.1710 - val_accuracy: 0.0093
Epoch 5/500
9/9 [==============================] - 1s 114ms/step - loss: 4.9775 - accuracy: 0.0140 - val_loss: 5.0767 - val_accuracy: 0.0093
Epoch 6/500
9/9 [==============================] - 1s 114ms/step - loss: 4.8781 - accuracy: 0.0143 - val_loss: 4.9944 - val_accuracy: 0.0093
Epoch 7/500
9/9 [==============================] - 1s 112ms/step - loss: 4.7719 - accuracy: 0.0148 - val_loss: 4.9224 - val_accuracy: 0.0110
Epoch 8/500
9

In [ ]:
idx2word = {v:k for k, v in word2idx.items()}
input2 = Input(shape=(1,))
x = embedding_layer(input2)
x, h, c = lstm(x, initial_state=[initial_h, initial_c])
output2 = dense(x)
sampling_model = Model([input2, initial_h, initial_c], [output2, h, c])
def print_line():
  np_input = np.array([[ word2idx['<sos>'] ]])
  h = np.zeros((1, 25))
  c = np.zeros((1, 25))
  eos = word2idx['<eos>']
  output_sentence = []
  for _ in range(max_sequence_length):
    o, h, c = sampling_model.predict([np_input, h, c])
    probs = o[0,0]
    probs[0] = 0
    probs /= probs.sum()
    idx = np.random.choice(len(probs), p=probs)
    if idx == eos:
      break
    output_sentence.append(idx2word.get(idx, 0))
    np_input[0,0] = idx
  return ' '.join(output_sentence)


In [ ]:
while True:
  for _ in range(10): # output a 10 word sentence
    print(print_line())
  ans = input("Type yes to print another line and no to end the program")
  if ans and ans[0].lower().startswith('n'):
    break

bricks, house from.
to make wagon brushing everywhere!—
he can't tell to that, 'if here you'd laces out. city slid
and seen a leaf that, and make simply 'going a-known
from no first strangeness
nature's times and the want.' boys like an brush leaves out.)
wanted had anything the worse ground but house
gnaw it's just smells not their substitute. when just the across the
hadn't they've got teased. it anyone on all will world, speak in
by the floor,
